## Title :
LSTM v/s GRU

## Description :
The goal of this exercise is to compare the performance between two popular gating methods, i.e LSTM and GRUs:

<img src="../fig/fig.png" style="width: 500px;">

## Instructions :
- Read the IMDB dataset from the helper code given.
- Take a quick look at your training inputs and labels.
- Pad the values to a fix number `max_words` in-order to have sequences of the same size.
- Build, compile and fit a GRU model
- Evaluate the model performance on the test set and report the test set accuracy.
- Again build, compile and fit a model but use the LSTM architecture instead. 
- Evaluate the LSTM model's performance on the test set and report the test set accuracy.
- Compare the performance of all the two models.

## Hints:

<a href="https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding" target="_blank">tf.keras.layers.Embedding()</a> Turns positive integers (indexes) into dense vectors of fixed size.

<a href="https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM" target="_blank">tf.keras.layers.LSTM()</a> Long Short-Term Memory layer - Hochreiter 1997.

<a href="https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense" target="_blank">tf.keras.layers.Dense()</a> Just your regular densely-connected NN layer.

## LSTM

We will use both GRU and LSTM to perform sentiment analysis in tensorflow.keras and compare their performance using the custom IMDB dataset.

In [1]:
# Import necessary libraries
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import RNN
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Input,Dense,LSTM,GRU,Embedding
from tensorflow.keras.preprocessing import sequence
from prettytable import PrettyTable
import pickle

2023-08-07 22:20:04.310432: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-07 22:20:04.336826: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-07 22:20:04.337460: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-07 22:20:04.934070: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [10]:
# We use the same dataset as the previous exercise 
# with open('imdb_mini.pkl','rb') as f:
    # X_train, y_train, X_test, y_test = pickle.load(f)


In [4]:
import pandas as pd
data = pd.read_csv("IMDB Dataset.csv")

import re

def process(x):
    x = re.sub('[,\.!?:()"]', '', x)
    x = re.sub('<.*?>', ' ', x)
    x = re.sub('http\S+', ' ', x)
    # \S+: matches one or more non-whitespace characters
    x = re.sub('[^a-zA-Z0-9]', ' ', x)
    # matches any character that is not (^ negation) an uppercase letter (A-Z), a lowercase letter (a-z), or a digit (0-9).
    x = re.sub('\s+', ' ', x)
    # \s: matches any whitespace character, such as spaces, tabs, and newline characters.
    return x.lower().strip()

data['review'] = data['review'].apply(lambda x: process(x))

import nltk
nltk.download('stopwords')
nltk.download('punkt')

sw_set = set(nltk.corpus.stopwords.words('english'))

def sw_remove(x):
    words = nltk.tokenize.word_tokenize(x)
    filtered_list = [word for word in words if word not in sw_set]
    return ' '.join(filtered_list)

data['review'] = data['review'].apply(lambda x: sw_remove(x))

from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size = 0.2, random_state=109)

X_train = train["review"]
y_train = 1 * (train["sentiment"] == "positive")
X_test = test["review"]
y_test = 1*(test["sentiment"] == "positive")

from keras.preprocessing.text import Tokenizer

dict_size = 5000
tokenizer = Tokenizer(num_words=dict_size)
tokenizer.fit_on_texts(data['review'])

train_rev_tokens = tokenizer.texts_to_sequences(X_train)
test_rev_tokens = tokenizer.texts_to_sequences(X_test)
seq_lengths =  np.array([len(sequence) for sequence in train_rev_tokens])

[nltk_data] Downloading package stopwords to /home/ting/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ting/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
# Similar to the previous exercise, we will pre-preprocess our review sequences
# We fix the vocabulary size to 5000 because our custom 
# dataset was curated with that
vocabulary_size = 5000
# Max word length for each review will be 500
max_words = 200
# we set the embedding size to 32
embedding_size=32
# Pre-padding sequences to max_words lenth
X_train = sequence.pad_sequences(train_rev_tokens, maxlen=max_words,padding='pre')
X_test = sequence.pad_sequences(test_rev_tokens, maxlen=max_words,padding='pre')

In [4]:
# We create the mapping between words and sequences
word2id = imdb.get_word_index()
# We need to adjust the mapping by 3 because of tensorflow.keras preprocessing
# more here: https://stackoverflow.com/questions/42821330/restore-original-text-from-keras-s-imdb-dataset
word2id = {k:(v+3) for k,v in word2id.items()}
word2id["<PAD>"] = 0
word2id["<START>"] = 1
word2id["<UNK>"] = 2
word2id["<UNUSED>"] = 3

# Reversing the key,value pair will give the id2word
id2word = {i: word for word, i in word2id.items()}

### ⏸ For this problem with `embedding_size=32` ($X_t$) and `hidden_size=100` ($H_{t-1}$), how many trainable weights are associated with the GRU Cell (assuming `use_bias=True`)?


#### A. 39600
#### B. 39800
#### C. 40200
#### D. 40400

In [5]:
### edTest(test_chow1) ###
# Submit an answer choice as a string below (eg. if you choose option A, put 'A')
answer1 = 'C'

In [6]:
# Comparing with GRU model
embedding_size=32
hidden_size = 100
gru_model=Sequential()
# Add Embedding, GRU and a Dense layer 
# Add Embedding layer with vocabulary_size, embedding_size and input_length
# Add GRU with hidden_size
# Add Dense layer with 1 unit and sigmoid activation
gru_model.add(Embedding(vocabulary_size, embedding_size, input_length=max_words))
gru_model.add(GRU(hidden_size))
gru_model.add(Dense(1, activation='sigmoid'))

gru_model.compile(loss='binary_crossentropy',optimizer = 'Adam', metrics=['accuracy'])

2023-08-10 15:03:06.045866: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [7]:
gru_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 32)           160000    
                                                                 
 gru (GRU)                   (None, 100)               40200     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 200301 (782.43 KB)
Trainable params: 200301 (782.43 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
### edTest(test_chow2) ###
gru_cnt_params = gru_model.count_params()

In [17]:
batch_size = 256
num_epochs = 3
gru_model.fit(X_train, y_train, batch_size=batch_size, epochs=num_epochs)
gru_score = gru_model.evaluate(X_test,y_test)
print(f'Model accuracy on the test set is {gru_score[1]:.2f}')

Epoch 1/3
  1/157 [..............................] - ETA: 20s - loss: 0.2224 - accuracy: 0.9219

2023-08-10 16:25:09.961027: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 32000000 exceeds 10% of free system memory.


157/157 [==============================] - 23s 144ms/step - loss: 0.2086 - accuracy: 0.9185
Epoch 2/3
157/157 [==============================] - 22s 143ms/step - loss: 0.1852 - accuracy: 0.9289
Epoch 3/3
313/313 [==============================] - 4s 12ms/step - loss: 0.3003 - accuracy: 0.8853
Model accuracy on the test set is 0.89


### ⏸ For this problem with `embedding_size=32` ($X_t$) and `hidden_size=100` ($H_{t-1}$), how many trainable weights are associated with the LSTM Cell (assuming `use_bias=True`)?


#### A. 52800
#### B. 53200
#### C. 54200
#### D. 51400

In [20]:
(32 + 100) * 100 * 4 + 100 * 4

53200

In [10]:
### edTest(test_chow3) ###
# Submit an answer choice as a string below (eg. if you choose option A, put 'A')
answer2 = 'A'

In [11]:
# Comparing with LSTM model
embedding_size=32
hidden_size = 100

lstm_model=Sequential()

# Add Embedding, LSTM and a Dense layer 
# Add Embedding layer with vocabulary_size, embedding_size and input_length
# Add LSTM with hidden_size
# Add Dense layer with 1 unit and sigmoid activation
lstm_model.add(Embedding(vocabulary_size, embedding_size, input_length=max_words))
lstm_model.add(LSTM(100))
lstm_model.add(Dense(1, activation='sigmoid'))

lstm_model.compile(loss='binary_crossentropy',optimizer = 'Adam', metrics=['accuracy'])

In [12]:
lstm_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 200, 32)           160000    
                                                                 
 lstm (LSTM)                 (None, 100)               53200     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 213301 (833.21 KB)
Trainable params: 213301 (833.21 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
### edTest(test_chow4) ###
lstm_cnt_params = lstm_model.count_params()

In [14]:
lstm_cnt_params

213301

In [15]:
batch_size = 256
num_epochs = 3
lstm_model.fit(X_train, y_train, batch_size=batch_size, epochs=num_epochs)
lstm_score = lstm_model.evaluate(X_test,y_test)
print(f'Model accuracy on the test set is {lstm_score[1]:.2f}')

Epoch 1/3


2023-08-10 16:11:33.029495: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 32000000 exceeds 10% of free system memory.


157/157 [==============================] - 27s 166ms/step - loss: 0.4546 - accuracy: 0.7897
Epoch 2/3
157/157 [==============================] - 26s 164ms/step - loss: 0.2467 - accuracy: 0.9018
Epoch 3/3
313/313 [==============================] - 5s 16ms/step - loss: 0.2680 - accuracy: 0.8911
Model accuracy on the test set is 0.89


In [16]:
# Finally, we compare the results from the three implementations above

pt = PrettyTable()
pt.field_names = ["Strategy","Test set accuracy"]
pt.add_row(["GRU RNN",f'{gru_score[1]*100:.2f}%'])
pt.add_row(["LSTM RNN",f'{lstm_score[1]*100:.2f}%'])
print(pt)

+----------+-------------------+
| Strategy | Test set accuracy |
+----------+-------------------+
| GRU RNN  |       88.94%      |
| LSTM RNN |       89.11%      |
+----------+-------------------+


### 🍲 Which variant is better, LSTM or GRU?

Both LSTM & GRUs solve the vanishing gradient problem of RNNs but each has their advantages and disadvantages. (Read [this](https://arxiv.org/pdf/1412.3555v1.pdf) paper for a thorough analysis of the two methods)
Based on your understanding, which architecture is more appropriate for the current analysis?

In [17]:
### edTest(test_chow5) ###
# Type your answer within in the quotes given
answer3 = 'LSTM'